### Goal ###
Find the nearest [California Data Exchange Center](https://cdec.water.ca.gov/) weather station to each of our California bins. This information will be used to assign each bin weather variable values.

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
%matplotlib inline
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


Since we have many bins which do not contain a weather monitoring station, the next step is to fill in their values from the nearest station. First find the nearest station to each bin. To do this we will use scipy.spatial.

In [19]:
bins = pd.read_csv('../data/spatial_data/california_bins.csv')
stations = pd.read_csv('../data/CDEC_weather_station_data/target_stations.csv')
stations.columns = ['station','elevation', 'lat', 'long']
stations = stations[stations.long != 0]
bin_array = np.column_stack([bins['long'], bins['lat']])
station_array = np.column_stack([stations['long'], stations['lat']])
stations.head()

,station,elevation,lat,long
0,AGW,1000,37.15100,-119.50500
1,ACN,2600,34.44600,-118.20000
2,ADM,6200,41.23700,-120.79200
3,AGP,9450,37.72663,-119.14173
4,ALD,923,40.23200,-123.71200


In [20]:
from scipy import spatial

station_tree = spatial.cKDTree(station_array)
dist, indexes = station_tree.query(bin_array)

nearest_station_names = []
for index in indexes:
    nearest_station_names.append(stations.iloc[index, 0])
    
bins['nearest_station_name'] = nearest_station_names
bins.head()

,long,lat,nearest_station_name
0,-116.98,33.02,ELC
1,-116.48,33.02,JUL
2,-115.98,33.02,FIS
3,-115.48,33.02,CAU
4,-114.98,33.02,CAU


Starting with the temperature dataset as a test case, now we will make an hourly time series spanning from 2006 to 2016 and assign each bin a temprature from it's nearest station each hour. 

In [21]:
# Annoying, but here goes...
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

temp_data = pd.read_csv("../data/training_data/weather_data/TEMP_1d.csv", parse_dates=['OBS_DATE'], usecols = ["STATION_ID", "OBS_DATE", "VALUE"], index_col = "OBS_DATE")
temp_data = temp_data[temp_data.VALUE != '---']
temp_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 30650 entries, 2015-01-01 23:00:00 to 2015-01-02 23:00:00
Data columns (total 2 columns):
STATION_ID    30650 non-null object
VALUE         30650 non-null int64
dtypes: int64(1), object(1)
memory usage: 718.4+ KB


Here is an example of a station which does not report regularly on the hour. I am sure there are many other cases of this in the dataset. There are also almost certainly missing values. To fix this, we will resample and interpolate to a regular hourly frequency over the whole dataset.

In [22]:
bud_station = temp_data[temp_data['STATION_ID'] == 'BUD']
bud_station.head()

,STATION_ID,VALUE
OBS_DATE,,
2015-01-01 23:00:00,BUD,33
2015-01-01 23:15:00,BUD,33
2015-01-01 23:30:00,BUD,33
2015-01-01 23:45:00,BUD,33
2015-01-02 00:00:00,BUD,33


In [23]:
bud_station = bud_station.drop_duplicates()
bud_station = bud_station.resample('min')
bud_station = bud_station.interpolate(method = 'linear')
bud_station = bud_station.resample('H')
bud_station = bud_station.interpolate(method = 'linear')
bud_station['STATION_ID'] = 'BUD'
bud_station.head()

,STATION_ID,VALUE
OBS_DATE,,
2015-01-01 23:00:00,BUD,33.000000
2015-01-02 00:00:00,BUD,32.555556
2015-01-02 01:00:00,BUD,32.111111
2015-01-02 02:00:00,BUD,31.800000
2015-01-02 03:00:00,BUD,31.533333


In [24]:
def regularize(group):
    group = group.drop_duplicates()
    group = group.resample('min')
    group = group.interpolate(method = 'linear')
    group = group.resample('H')
    group = group.interpolate(method = 'linear')
    return group
    
grouped_temp_data = temp_data.groupby('STATION_ID')
st_regularized_temp_data = grouped_temp_data.apply(regularize)

st_regularized_temp_data['STATION_ID'] = st_regularized_temp_data.index.get_level_values(0)
st_regularized_temp_data = st_regularized_temp_data.reset_index(level = 0, drop = True)
st_regularized_temp_data = st_regularized_temp_data.set_index(['STATION_ID'], append = True)
st_regularized_temp_data.head()

,,VALUE
OBS_DATE,STATION_ID,
2015-01-01 23:00:00,ACN,31.0
2015-01-02 00:00:00,ACN,30.0
2015-01-02 01:00:00,ACN,29.0
2015-01-02 02:00:00,ACN,31.0
2015-01-02 03:00:00,ACN,33.0


Parallelize it?!

In [25]:
n_threads = 14

def regularize(group):
    group = group.drop_duplicates()
    group = group.resample('min')
    group = group.interpolate(method = 'linear')
    group = group.resample('H')
    group = group.interpolate(method = 'linear')
    return group

def group_data(temp_data):
    grouped_data = temp_data.groupby('STATION_ID')
    return grouped_data.apply(regularize)
    
def parallelize(temp_data, func, n_cores = n_threads):
    '''Parallelizes regularization, takes temp data and
    splits up regularization fuction over avalibile threads'''
    temp_data_split = np.array_split(temp_data, n_cores)
    pool = Pool(n_cores)
    result = pd.concat(pool.map(func, temp_data_split))
    pool.close()
    pool.join()
    return result
    
mt_regularized_temp_data = parallelize(temp_data, group_data)

mt_regularized_temp_data['STATION_ID'] = mt_regularized_temp_data.index.get_level_values(0)
mt_regularized_temp_data = mt_regularized_temp_data.reset_index(level = 0, drop = True)
mt_regularized_temp_data = mt_regularized_temp_data.set_index(['STATION_ID'], append = True)
mt_regularized_temp_data.head()

,,VALUE
OBS_DATE,STATION_ID,
2015-01-01 23:00:00,ACN,31.0
2015-01-02 00:00:00,ACN,30.0
2015-01-02 01:00:00,ACN,29.0
2015-01-02 02:00:00,ACN,31.0
2015-01-02 03:00:00,ACN,33.0


This next big block uses resampling to bin the temp. data at a resolution of six hours. In early runs I had memory issues so I added this. Hoping I won't need it in the final product. Keeping the code here I case I want to use it again.

In [26]:
# n_threads = 14

# def downsample_timeseries(group):
#     group = group.sort_index()
#     group.loc[:,'resampled_value'] = group.VALUE.rolling('6H').mean()
#     group = group.drop(['VALUE'], axis = 1)
#     return group.iloc[0::12, :]

# def group_timeseries_data(stations):
#     data = temp_data.loc[temp_data['STATION_ID'].isin(stations)]
#     grouped_data = data.groupby('STATION_ID')
#     return grouped_data.apply(downsample_timeseries)
    
# def parallelize(stations, func, n_cores = n_threads):
#     '''Parallelizes downsampling, takes list of stations and
#     splits up the downsampling fuction over avalibile threads'''
#     stations_split = np.array_split(stations, n_cores)
#     pool = Pool(n_cores)
#     result = pd.concat(pool.map(func, stations_split))
#     pool.close()
#     pool.join()
#     return result
    
# stations = temp_data['STATION_ID'].unique().tolist()
# binned_temp_data = parallelize(stations, group_timeseries_data)
# binned_temp_data = binned_temp_data.reset_index(level = 0, drop = True)
# binned_temp_data = binned_temp_data.set_index(['STATION_ID'], append = True)
# binned_temp_data.head()

In [27]:
#binned_temp_data.to_csv('../data/training_data/weather_data/TEMP_1yr_binned_6hr.csv')

In [28]:
time_series = pd.date_range("2015-01-01 23:00:00", "2015-01-02 23:00:00", freq = "H")
time_series = time_series.to_series()

Now we make a dataframe of 'noxels' or n-dimentional voxels. Each row will be a bin at a specific time. Once we have this dataframe, we can go back though and assign weather variable values to each noxel.

In [29]:
def apply_to_bins(time_series, bins):
    return bins.assign(time = time_series)

%memit noxels = pd.concat(time_series.apply(apply_to_bins, args = (bins,)).tolist())
noxels.to_csv('../data/training_data/noxels.csv', index = False)
noxels.head()

peak memory: 213.32 MiB, increment: -0.25 MiB


,long,lat,nearest_station_name,time
0,-116.98,33.02,ELC,2015-01-01 23:00:00
1,-116.48,33.02,JUL,2015-01-01 23:00:00
2,-115.98,33.02,FIS,2015-01-01 23:00:00
3,-115.48,33.02,CAU,2015-01-01 23:00:00
4,-114.98,33.02,CAU,2015-01-01 23:00:00


In [32]:
mt_regularized_temp_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 21100 entries, (2015-01-01 23:00:00, ACN) to (2015-01-02 23:00:00, WWS)
Data columns (total 1 columns):
VALUE    21090 non-null float64
dtypes: float64(1)
memory usage: 891.5+ KB


In [34]:
st_regularized_temp_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 11105 entries, (2015-01-01 23:00:00, ACN) to (2015-01-02 19:00:00, ZEN)
Data columns (total 1 columns):
VALUE    11103 non-null float64
dtypes: float64(1)
memory usage: 464.1+ KB


In [15]:
n_threads = 14

def try_except(row):
    try:
        return regularized_temp_data.loc[row['time'], row['nearest_station_name']]
    except:
        return np.nan
    
def add_data(noxels):
    noxels['temp'] = noxels.apply(lambda row: try_except(row), axis = 1)
    return noxels

def parallelize(noxels, func, n_cores = n_threads):
    '''Parallelizes downsampling, takes list of stations and
    splits up the downsampling fuction over avalibile threads'''
    noxels_split = np.array_split(noxels, n_cores)
    pool = Pool(n_cores)
    result = pd.concat(pool.map(func, noxels_split))
    pool.close()
    pool.join()
    return result

noxels = parallelize(noxels, add_data)
noxels = noxels.dropna()
noxels['temp'] = noxels['temp']
noxels.to_csv('../data/training_data/noxels_TEMP.csv', index = False)
noxels.head()

,long,lat,nearest_station_name,time,temp
1,-116.48,33.02,JUL,2015-01-01 23:00:00,[23.0]
2,-115.98,33.02,FIS,2015-01-01 23:00:00,[45.0]
3,-115.48,33.02,CAU,2015-01-01 23:00:00,[39.0]
4,-114.98,33.02,CAU,2015-01-01 23:00:00,[39.0]
5,-117.48,33.52,BEC,2015-01-01 23:00:00,[42.0]


In [16]:
noxels.to_csv('../data/training_data/noxels.csv', index = False)

OK, so, working to a first approximation. I am worried that we a loosing a bunch of data somewhere. Lets check some lengths and see. If we filled every bin with a temperature value, then we should have bins x time_series rows in our data file...

In [17]:
(len(noxels) / (len(bins) * len(time_series))) * 100

67.70186335403726